In [ ]:
import os
import subprocess
import time
import sys
import warnings


print("Setting up...")
!pip install -q -U langgraph langchain-ollama langchain-community duckduckgo-search==5.3.1 ddgs

if not os.path.exists("/usr/local/bin/ollama"):
    !curl -fsSL https://ollama.com/install.sh | sh

try:
    subprocess.check_output(["curl", "-s", "http://localhost:11434"])
except:
    subprocess.Popen("nohup ollama serve > server.log 2>&1 &", shell=True)
    time.sleep(5)

print("Verifying Model...")
!ollama pull llama3.1


In [31]:


import warnings
warnings.filterwarnings("ignore")

from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import SystemMessage, HumanMessage

print("\nINITIALIZING BOT...")


llm = ChatOllama(model="llama3.1", temperature=0)

@tool
def web_search(query: str):
    try:
        from langchain_community.tools import DuckDuckGoSearchRun
        search = DuckDuckGoSearchRun()
        return search.invoke(query)
    except:
        return "Search failed."

tools = [web_search]

sys_msg = SystemMessage(content="""
****Give short casual answers with below rules.
-RULES:
-You are a smart AI assistant and you are supposed to be talking in the most human way possible and do not assume anything about user or whatever casual thing user tells for a fact. For example, if user tells you his/her name then dont search it up and assume that it's him/her beacuse maybe he is just having a casual conversation and later he may ask something. Just talk normally like a human being.
-Do not search on web for making a statement as "Searching for web" just for a fact or something related to what has already been told to you isnt human like behaviour.
-Greetings or general conversations should be taken care of without searching memory or web. If user is introducing himself or telling something about himself or somehting that is a fact just remember it and answer like a normal human conversation. And if it is a part memory based statement and partly a question needed to be searched then look for the keywords that just need to be searched and then look into the memory and answer accordingly.
-**Memory First:** If user is having a general conversation or has asked you a mathematical/logical/reasoning/analytocal question answer it without searching on web. If you think user is asking something that is personal and he did not talk about it before then say you dont know it.
-Before searching on web just think if you really need to check. That is, if you feel user is going to say something in the next statement then maybe search is not needed but extra information on his statement is needed so just ask about it nicely without searching the web.
-**External Facts:** Use 'web_search' for any of the questions user poses that has not been in your memory and is something that you are not trained for or if it is current affair or a fact or any news or anything happening in thre world that you dont have prior knowledge about.
""")

memory = MemorySaver()
graph = create_react_agent(llm, tools=tools, checkpointer=memory)

print("\nBOT READY! (Final Production Mode)")
print("-" * 50)

config = {"configurable": {"thread_id": "Final-Session"}}

while True:
    try:
        user_input = input("\nYou: ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("Goodbye!")
            break

        print("Bot: ", end="", flush=True)

        inputs = {"messages": [sys_msg, HumanMessage(content=user_input)]}

        for event in graph.stream(inputs, config=config, stream_mode="values"):
            message = event["messages"][-1]

            if message.type == "ai" and message.content:
                if not message.tool_calls:
                    print(message.content, end="", flush=True)


        print("")

    except Exception as e:
        print(f"\nError: {e}")

⚙️ Setting up...
⬇️ Verifying Model...


🤖 INITIALIZING BOT...

✅ BOT READY! (Final Production Mode)
--------------------------------------------------

You: Hello
Bot: Hello! Nice to meet you. Is there something I can help you with?

You: My name is Siddharth Singh and I am doing fine.
Bot: Nice to meet you, Siddharth! It's great to hear that you're doing fine. What brings you here today? Do you have any questions or topics you'd like to discuss?

You: Yes, actually
Bot: So you were saying "yes, actually"? That's an interesting phrase! I think I understand what you're getting at. You want to emphasize or clarify something, right? Would you like to talk more about that?

You: What is the news on Indigo flights getting cancelled.
Bot: It seems like there's been a significant issue with Indigo flights getting cancelled. The airline has faced a major operational crisis, cancelling over 850 flights in the past few days due to staff shortages and new crew regulations. The aviation ministry 

KeyboardInterrupt: Interrupted by user